In [ ]:
import os

import polars as pl

from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor

import kaggle_evaluation.mcts_inference_server
from pathlib import Path

# Path to competition files
comp_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants')

target = 'utility_agent1'
counter = 0

# Load training data
train = pl.read_csv(comp_path / 'train.csv')
y_train = train[target]

cols_to_drop = ['num_draws_agent1', 'num_losses_agent1', 'num_wins_agent1', target]
train = train.drop(cols_to_drop)

# Encode categorical columns
obj_cols = train.select(pl.col(pl.String)).columns
enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-999, encoded_missing_value=-9999)
enc.fit(train[obj_cols])

train_transformed = enc.transform(train[obj_cols])
for e, c in enumerate(obj_cols):
    train = train.with_columns(pl.Series(c, train_transformed[:, e]))

# Define models to test with Bagging
models = {
    'rf': RandomForestRegressor(),
    'gbr': GradientBoostingRegressor(),
}

# Define parameter grids for grid search
param_grid = {
    'rf': {'n_estimators': [100, 200], 'max_depth': [5, 10]},
    'gbr': {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1]},
}

# Perform GridSearchCV for each model
best_models = {}
for model_name in models:
    grid = GridSearchCV(BaggingRegressor(base_estimator=models[model_name], n_jobs=-1),
                        param_grid[model_name], cv=3, scoring='neg_mean_squared_error')
    grid.fit(train, y_train)
    best_models[model_name] = grid.best_estimator_

    # Output best parameters
    print(f"Best parameters for {model_name}: {grid.best_params_}")

# Select the best model based on cross-validation score
best_model_name = max(best_models, key=lambda m: -grid.best_score_)
best_model = best_models[best_model_name]
print(f"Best model: {best_model_name}")

# Save the best model
joblib.dump(best_model, 'best_model.joblib')


In [ ]:
import joblib

counter = 0
best_model = None

# Prediction function
def predict(test: pl.DataFrame, submission: pl.DataFrame):
    global counter, best_model
    if counter == 0:
        # Load the pre-trained model during the first call
        best_model = joblib.load('best_model.joblib')

    counter += 1

    # Transform test data
    test_transformed = enc.transform(test[obj_cols])
    for e, c in enumerate(obj_cols):
        test = test.with_columns(pl.Series(c, test_transformed[:, e]))

    # Make predictions
    predictions = best_model.predict(test)

    # Update the submission DataFrame with predictions
    submission = submission.with_columns(pl.Series(target, predictions))

    # Save the predictions as 'submission.parquet'
    submission.write_parquet('submission.parquet')

    # Return the submission DataFrame
    return submission

# Initialize the inference server
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )

In [ ]:
import os
import polars as pl
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestRegressor

import kaggle_evaluation.mcts_inference_server
from pathlib import Path

# Path to competition files
comp_path = Path('/kaggle/input/um-game-playing-strength-of-mcts-variants')

target = 'utility_agent1'
counter = 0

# Model training function
def train_model():
    global obj_cols, enc, rf

    # Load training data
    train = pl.read_csv(comp_path / 'train.csv')
    y_train = train[target]

    # Drop unnecessary columns
    cols_to_drop = ['num_draws_agent1', 'num_losses_agent1', 'num_wins_agent1', target]
    train = train.drop(cols_to_drop)
    
    # Encode categorical columns
    obj_cols = train.select(pl.col(pl.String)).columns
    enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-999, encoded_missing_value=-9999)
    enc.fit(train[obj_cols])
    
    # Transform the categorical data
    train_transformed = enc.transform(train[obj_cols])
    for e, c in enumerate(obj_cols):
        train = train.with_columns(pl.Series(c, train_transformed[:, e]))
    
    # Train Random Forest Regressor
    rf = RandomForestRegressor(n_estimators=100, max_depth=5, n_jobs=-1)
    rf.fit(train, y_train)

# Prediction function
def predict(test: pl.DataFrame, submission: pl.DataFrame):
    global counter
    if counter == 0:
        # Perform model training during the first call
        train_model()
    counter += 1

    # Transform test data
    test_transformed = enc.transform(test[obj_cols])
    for e, c in enumerate(obj_cols):
        test = test.with_columns(pl.Series(c, test_transformed[:, e]))

    # Make predictions
    predictions = rf.predict(test)

    # Update the submission DataFrame with predictions
    submission = submission.with_columns(pl.Series(target, predictions))

    # Save the predictions as 'submission.parquet'
    submission.write_parquet('submission.parquet')

    # Return the submission DataFrame
    return submission

# Initialize the inference server
inference_server = kaggle_evaluation.mcts_inference_server.MCTSInferenceServer(predict)

if os.getenv('KAGGLE_IS_COMPETITION_RERUN'):
    inference_server.serve()
else:
    inference_server.run_local_gateway(
        (
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/test.csv',
            '/kaggle/input/um-game-playing-strength-of-mcts-variants/sample_submission.csv'
        )
    )


In [ ]:
#Describe showing Only the requested statistics (mean, minimum and maximum). Then, transpose the table.

df.describe().loc[['mean','min','max']].T